In [1]:
from torch import nn
from collections.abc import Sequence

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from utils import load_model

from monai.networks.blocks.segresnet_block import ResBlock, get_conv_layer, get_upsample_layer
from monai.networks.layers.factories import Dropout
from monai.networks.layers.utils import get_act_layer, get_norm_layer
from monai.utils import UpsampleMode
from typing import Union, Tuple, List, Dict, Optional
import json

In [15]:
class SegResNet(nn.Module):
    def __init__(
        self,
        spatial_dims: int = 3,
        init_filters: int = 8,
        in_channels: int = 1,
        out_channels: int = 2,
        dropout_prob: Union[float, None] = None,
        act: Union[Tuple, str] = ("RELU", {"inplace": True}),
        norm: Union[Tuple, str] = ("GROUP", {"num_groups": 8}),
        norm_name: str = "",
        num_groups: int = 8,
        use_conv_final: bool = True,
        blocks_down: Tuple[int, ...] = (1, 2, 2, 4),
        blocks_up: Tuple[int, ...] = (1, 1, 1),
        upsample_mode: Union[UpsampleMode, str] = UpsampleMode.NONTRAINABLE,
    ):
        super().__init__()

        if spatial_dims not in (2, 3):
            raise ValueError("`spatial_dims` can only be 2 or 3.")

        self.spatial_dims = spatial_dims
        self.init_filters = init_filters
        self.in_channels = in_channels
        self.blocks_down = blocks_down
        self.blocks_up = blocks_up
        self.dropout_prob = dropout_prob
        self.act = act  # input options
        self.act_mod = get_act_layer(act)
        if norm_name:
            if norm_name.lower() != "group":
                raise ValueError(f"Deprecating option 'norm_name={norm_name}', please use 'norm' instead.")
            norm = ("group", {"num_groups": num_groups})
        self.norm = norm
        self.upsample_mode = UpsampleMode(upsample_mode)
        self.use_conv_final = use_conv_final
        self.convInit = get_conv_layer(spatial_dims, in_channels, init_filters)
        self.down_layers = self._make_down_layers()
        self.up_layers, self.up_samples = self._make_up_layers()
        self.conv_final = self._make_final_conv(out_channels)

        if dropout_prob is not None:
            self.dropout = Dropout[Dropout.DROPOUT, spatial_dims](dropout_prob)

    def _make_down_layers(self):
        down_layers = nn.ModuleList()
        blocks_down, spatial_dims, filters, norm = (self.blocks_down, self.spatial_dims, self.init_filters, self.norm)
        for i, item in enumerate(blocks_down):
            layer_in_channels = filters * 2**i
            pre_conv = (
                get_conv_layer(spatial_dims, layer_in_channels // 2, layer_in_channels, stride=2)
                if i > 0
                else nn.Identity()
            )
            down_layer = nn.Sequential(
                pre_conv, *[ResBlock(spatial_dims, layer_in_channels, norm=norm, act=self.act) for _ in range(item)]
            )
            down_layers.append(down_layer)
        return down_layers

    def _make_up_layers(self):
        up_layers, up_samples = nn.ModuleList(), nn.ModuleList()
        upsample_mode, blocks_up, spatial_dims, filters, norm = (
            self.upsample_mode,
            self.blocks_up,
            self.spatial_dims,
            self.init_filters,
            self.norm,
        )
        n_up = len(blocks_up)
        for i in range(n_up):
            sample_in_channels = filters * 2 ** (n_up - i)
            up_layers.append(
                nn.Sequential(
                    *[
                        ResBlock(spatial_dims, sample_in_channels // 2, norm=norm, act=self.act)
                        for _ in range(blocks_up[i])
                    ]
                )
            )
            up_samples.append(
                nn.Sequential(
                    *[
                        get_conv_layer(spatial_dims, sample_in_channels, sample_in_channels // 2, kernel_size=1),
                        get_upsample_layer(spatial_dims, sample_in_channels // 2, upsample_mode=upsample_mode),
                    ]
                )
            )
        return up_layers, up_samples

    def _make_final_conv(self, out_channels: int):
        return nn.Sequential(
            get_norm_layer(name=self.norm, spatial_dims=self.spatial_dims, channels=self.init_filters),
            self.act_mod,
            get_conv_layer(self.spatial_dims, self.init_filters, out_channels, kernel_size=1, bias=True),
        )

    def encode(self, x: torch.Tensor) -> Tuple[torch.Tensor, List[torch.Tensor]]:
        x = self.convInit(x)
        if self.dropout_prob is not None:
            x = self.dropout(x)

        down_x = []

        for down in self.down_layers:
            x = down(x)
            down_x.append(x)

        return x, down_x

    def decode(self, x: torch.Tensor, down_x: List[torch.Tensor]) -> torch.Tensor:
        for i, (up, upl) in enumerate(zip(self.up_samples, self.up_layers)):
            x = up(x) + down_x[i + 1]
            x = upl(x)

        if self.use_conv_final:
            x = self.conv_final(x)

        return x


    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x, down_x = self.encode(x)
        down_x.reverse()

        x = self.decode(x, down_x)
        return x
    

class MobileWrapper(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, x):
        # RBG to grayscale
        x = torch.mean(x, dim=1, keepdim=True)
        # add batch dim
        # x = x.unsqueeze(0)

        x = self.model(x)

        res : Dict[str, torch.Tensor] = {}
        res["out"] = x
        return res



In [9]:
run_name = "2023-11-08_19-49-52"

run_path = f"runs/{run_name}/"

train_summary = json.load(open(run_path + "train_summary.json"))

model_name = train_summary["config"]["MODEL"]
IMAGE_SIZE = train_summary["config"]["IMAGE_SIZE"]

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [16]:
import torch
import torchvision
from torch.utils.mobile_optimizer import optimize_for_mobile



model = SegResNet(in_channels=1, out_channels=2, spatial_dims=2)
model = load_model(model, run_path + "best_model.pth")

model_mobile = MobileWrapper(model)

In [17]:
model_mobile.eval()
example = torch.rand(1, 3, 256, 256)

out = model_mobile(example)
print(out)

{'out': tensor([[[[ 1.0446,  1.0462,  0.9783,  ...,  0.9745,  0.9883,  1.0939],
          [ 1.0527,  1.1909,  1.1640,  ...,  1.0258,  1.0346,  1.0487],
          [ 1.3599,  1.2263,  1.4739,  ...,  1.0556,  0.9517,  0.9987],
          ...,
          [ 1.9674,  2.0573,  1.8549,  ...,  1.7608,  1.6719,  0.8930],
          [ 2.0589,  2.0807,  2.1081,  ...,  1.9305,  1.7167,  0.8926],
          [ 1.7014,  1.6551,  1.6030,  ...,  1.6165,  1.2686,  0.8988]],

         [[-3.2444, -3.2493, -3.0274,  ..., -3.0148, -3.0919, -3.4127],
          [-3.2707, -3.2028, -2.7732,  ..., -3.1828, -3.2115, -3.2644],
          [-3.1608, -2.7303, -2.8102,  ..., -3.2803, -2.9402, -3.0942],
          ...,
          [-3.1360, -3.0108, -2.5942,  ..., -2.6833, -3.0548, -2.7484],
          [-3.6295, -3.1460, -3.2558,  ..., -2.9810, -3.2235, -2.7470],
          [-3.4532, -3.3819, -3.1156,  ..., -3.1775, -3.0324, -2.7674]]]],
       grad_fn=<ConvolutionBackward0>)}


In [18]:
model_mobile.eval()
example = torch.rand(1, 3, 256, 256)
# traced_module = torch.jit.trace(model, example, strict=False)
traced_script_module = torch.jit.script(model_mobile)
traced_script_module_optimized = optimize_for_mobile(traced_script_module)
traced_script_module_optimized._save_for_lite_interpreter("model.ptl")


In [19]:
from torch.jit.mobile import (
    _backport_for_mobile,
    _get_model_bytecode_version,
)

print(_get_model_bytecode_version("model.ptl"))

_backport_for_mobile("model.ptl", "model_7.ptl", 7)

print(_get_model_bytecode_version("model_7.ptl"))

8
7


In [7]:
import torch
from torch.utils.mobile_optimizer import optimize_for_mobile

model = torch.hub.load('pytorch/vision:v0.11.0', 'deeplabv3_resnet50', pretrained=True)
model.eval();

scripted_module = torch.jit.script(model)
optimized_scripted_module = optimize_for_mobile(scripted_module)

# Export full jit version model (not compatible with lite interpreter)
scripted_module.save("deeplabv3_scripted.pt")
# Export lite interpreter version model (compatible with lite interpreter)
scripted_module._save_for_lite_interpreter("deeplabv3_scripted.ptl")
# using optimized lite interpreter model makes inference about 60% faster than the non-optimized lite interpreter model, which is about 6% faster than the non-optimized full jit model
optimized_scripted_module._save_for_lite_interpreter("deeplabv3_scripted_optimized.ptl")

Using cache found in /home/rikhat.akizhanov/.cache/torch/hub/pytorch_vision_v0.11.0
/home/rikhat.akizhanov/Desktop/old_torch/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/rikhat.akizhanov/Desktop/old_torch/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
img = torch.rand(1, 3, 256, 256)
out = model(img)

print(out["out"].shape)



torch.Size([1, 21, 256, 256])
